Our training set wants to detect ecstasy pills vs. not pills.  
Dropping a few records is not a massive problem

In [ ]:
import pandas as pd 
import numpy as np
import logging

In [ ]:
# Display progress logs on stdout
logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s %(levelname)s %(message)s')

In [ ]:
df = pd.read_csv("data_for_model.csv", low_memory=False)

In [ ]:
# Binary classifier - need to detect
df_dedup = df.drop_duplicates("category_concat_cat")
f1 = df_dedup["category_concat_cat"].str.lower().str.contains("ecsta")
f2 = df_dedup["category_concat_cat"].str.lower().str.contains("pill")
ccc = list(df_dedup[f1&f2]["category_concat_cat"])
df.loc[df.category_concat_cat.isin(ccc), "y"] = 1
df.loc[~(df.category_concat_cat.isin(ccc)), "y"] = 0

df = df[pd.notnull(df["listing_text"])]
y = df["y"]
x = df["listing_text"]
# df.pivot_table(columns = "y", values="listing_text", aggfunc=lambda x: len(x))


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.25)

Learn vocabulary and idf, return term-document matrix.

- The 'fit' part computes the term freq/inverse doc freq weightings
- The 'transform' part then turns the X_train documents into vectors

The resultant table looks like this (but is stored as a sparse array)

| index | term1 | term2 | etc |
|-------|-------|-------|-----|
| 0     | 0.000 | 0.707 |     |
| 1     | 0.212 | 0.621 |     |
| 2     | 0.457 | 0.000 |     |
| etc   |       |       |     |

In [ ]:
# Convert a collection or raw documents into a matrix of TD-IDF features
from sklearn.feature_extraction.text import TfidfVectorizer

# Vectorizer is an object that we can then pass data into and it will transform it appropriately
vectorizer = TfidfVectorizer(sublinear_tf=True, max_df=0.5,
                                 stop_words='english')
X_train_v = vectorizer.fit_transform(X_train)

# Now we've fitted using the training data, we can use the same tdf/idf weightings on the test set
X_test_v = vectorizer.transform(X_test)

In [ ]:
from sklearn.feature_selection import SelectKBest, chi2
feature_names = vectorizer.get_feature_names()
ch2 = SelectKBest(chi2, k=300)
X_train = ch2.fit_transform(X_train_v, y_train)
feature_names = [feature_names[i] for i in ch2.get_support(indices=True)]